In [4]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
from scipy.signal import stft
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
import matplotlib.pyplot as plt

# Load the data
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\new.mat'
data = sio.loadmat(file_path)
eeg_data = data['EEG']
labels = data['label_list']

# Reshape EEG data to have samples as the first dimension
eeg_data_reshaped = np.transpose(eeg_data, (2, 1, 0))

# Apply STFT to EEG data
_, _, stft_data = stft(eeg_data_reshaped, axis=1)
stft_data = np.abs(stft_data)

# Reshape the STFT data for CNN input (samples, height, width, channels)
stft_data_reshaped = stft_data.reshape(stft_data.shape[0], stft_data.shape[1], stft_data.shape[2], 1)

# Flatten the labels array
labels_flat = labels.flatten()

# Apply SMOTE to balance the classes
# Flatten the STFT data for SMOTE application
stft_data_flattened = stft_data_reshaped.reshape(stft_data_reshaped.shape[0], -1)

# Apply SMOTE
smote = SMOTE(random_state=42)
stft_data_resampled, labels_resampled = smote.fit_resample(stft_data_flattened, labels_flat)

# Reshape back to the original shape for CNN input
stft_data_resampled = stft_data_resampled.reshape(-1, stft_data.shape[1], stft_data.shape[2], 1)

# Check the shape of resampled data and labels
print("Shape of resampled data:", stft_data_resampled.shape)
print("Shape of resampled labels:", labels_resampled.shape)





ValueError: cannot reshape array of size 108510672 into shape (11068,129,19,1)

In [3]:

# Split the resampled data and labels into training and test sets
X_train, X_test, y_train, y_test = train_test_split(stft_data_resampled, labels_resampled, test_size=0.2, random_state=42)

# Verify the shapes after splitting
print("Shape of training data:", X_train.shape)
print("Shape of training labels:", y_train.shape)
print("Shape of test data:", X_test.shape)
print("Shape of test labels:", y_test.shape)

# Build the CNN model for feature extraction
input_shape = (X_train.shape[1], X_train.shape[2], 1)

cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D((2, 2)))
cnn_model.add(Flatten())

# Create a model to output the flattened features
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.output)

# Extract features from training and test data
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

# Train ensemble classifier on the extracted features
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(probability=True, random_state=42)

ensemble_clf = VotingClassifier(estimators=[('rf', rf_clf), ('svm', svm_clf)], voting='soft')
ensemble_clf.fit(X_train_features, y_train)
# Make predictions and evaluate the model
y_pred = ensemble_clf.predict(X_test_features)
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, ensemble_clf.predict_proba(X_test_features)[:, 1])

print("Accuracy:", accuracy)
print("ROC AUC:", roc_auc)

# Plot ROC Curve
fpr, tpr, _ = roc_curve(y_test, ensemble_clf.predict_proba(X_test_features)[:, 1])
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

NameError: name 'stft_data_reshaped' is not defined

In [1]:
import numpy as np
import scipy.io as sio
from scipy.stats import kurtosis, skew
from scipy.signal import welch
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, roc_curve, auc, make_scorer
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, LSTM
import optuna
import pywt
from xgboost import XGBClassifier

# Load the data
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\new.mat'
new = sio.loadmat(file_path)
Label = new['label_list'].flatten()
EEG = new['EEG']
ET = new['ET']

# Feature extraction for ML (EEG)
def extract_ml_features(data):
    features = []
    for i in range(data.shape[2]):  # Iterate over samples
        sample = data[:, :, i]
        sample_features = []
        for j in range(data.shape[0]):  # Iterate over channels
            channel_data = sample[j, :]
            # Statistical features
            mean = np.mean(channel_data)
            var = np.var(channel_data)
            skewness = skew(channel_data)
            kurt = kurtosis(channel_data)
            # Frequency domain features using Welch's method
            freqs, psd = welch(channel_data)
            psd_mean = np.mean(psd)
            psd_std = np.std(psd)
            # Combine all features
            sample_features.extend([mean, var, skewness, kurt, psd_mean, psd_std])
        features.append(sample_features)
    return np.array(features)

# Wavelet transform features (EEG)
def extract_wavelet_features(data):
    features = []
    for i in range(data.shape[2]):  # Iterate over samples
        sample = data[:, :, i]
        sample_features = []
        for j in range(data.shape[0]):  # Iterate over channels
            channel_data = sample[j, :]
            coeffs = pywt.wavedec(channel_data, 'db4', level=4)
            for coeff in coeffs:
                sample_features.extend([np.mean(coeff), np.std(coeff)])
        features.append(sample_features)
    return np.array(features)

ml_features = extract_ml_features(EEG)
wavelet_features = extract_wavelet_features(EEG)
ml_features = np.concatenate((ml_features, wavelet_features), axis=1)

# Feature extraction for DL (EEG) using CNN and LSTM
input_shape = (EEG.shape[0], EEG.shape[1], 1)

input_layer = Input(shape=input_shape)
conv1 = Conv2D(32, (3, 3), activation='relu')(input_layer)
pool1 = MaxPooling2D((2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)
flatten = Flatten()(pool2)

# Reshape for LSTM layer
reshape_layer = Reshape((flatten.shape[1], 1))(flatten)
lstm_layer = LSTM(64)(reshape_layer)
dense1 = Dense(128, activation='relu')(lstm_layer)
output_layer = Dense(64, activation='relu')(dense1)  # Output for feature extraction

cnn_rnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_rnn_model.compile(optimizer='adam', loss='mse')

# Reshape data for CNN input
data_cnn = EEG.reshape(EEG.shape[2], EEG.shape[0], EEG.shape[1], 1)

cnn_rnn_features = cnn_rnn_model.predict(data_cnn)

# Combine ML and DL features (EEG)
combined_eeg_features = np.concatenate((ml_features, cnn_rnn_features), axis=1)

# Feature extraction (ET)
image_list = []
handcrafted_features = []

for sample_idx in range(ET.shape[2]):
    x_left, y_left, pupil_left, x_right, y_right, pupil_right = ET[:, :, sample_idx]
    
    gaze_plot = np.zeros((64, 64, 3), dtype=np.uint8)  # Initialize as black image

    scaled_x_left = (x_left * 1920 / 30).astype(int)  # 1920 / 30 ~ 64
    scaled_y_left = (y_left * 1080 / 16.875).astype(int)  # 1080 / 16.875 ~ 64
    scaled_x_right = (x_right * 1920 / 30).astype(int)
    scaled_y_right = (y_right * 1080 / 16.875).astype(int)

    for i in range(120):
        if 0 <= scaled_x_left[i] < 64 and 0 <= scaled_y_left[i] < 64:
            gaze_plot[scaled_y_left[i], scaled_x_left[i]] = [255, 255, 255]  # White color
        if 0 <= scaled_x_right[i] < 64 and 0 <= scaled_y_right[i] < 64:
            gaze_plot[scaled_y_right[i], scaled_x_right[i]] = [255, 255, 255]  # White color

    image_list.append(gaze_plot)

    mean_left = np.mean([x_left, y_left, pupil_left])
    std_left = np.std([x_left, y_left, pupil_left])
    mean_right = np.mean([x_right, y_right, pupil_right])
    std_right = np.std([x_right, y_right, pupil_right])
    
    handcrafted_features.append([mean_left, std_left, mean_right, std_right])

image_array = np.array(image_list)
handcrafted_features = np.array(handcrafted_features)

# Reshape images to have a single channel (grayscale)
image_array = np.expand_dims(image_array, axis=-1)
image_array = image_array / 255.0  # Normalize the image data to the range [0, 1]

# LeNet-5 for feature extraction (ET)
lenet5 = tf.keras.Sequential([
    Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, kernel_size=(5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten()
])

lenet5_features = lenet5.predict(image_array)

# Combine handcrafted features with LeNet-5 features (ET)
et_combined_features = np.concatenate((lenet5_features, handcrafted_features), axis=1)

# Combine EEG and ET features
combined_features = np.concatenate((combined_eeg_features, et_combined_features), axis=1)

# Handle imbalanced data using SMOTE
smote = SMOTE(random_state=42)
combined_features_resampled, labels_resampled = smote.fit_resample(combined_features, Label)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(combined_features_resampled, labels_resampled, test_size=0.2, random_state=42)

# Best parameters found by Optuna (previously optimized)
best_params = {'rf_n_estimators': 265, 'gb_n_estimators': 89, 'xgb_n_estimators': 300}

rf_clf = RandomForestClassifier(n_estimators=best_params['rf_n_estimators'])
gb_clf = GradientBoostingClassifier(n_estimators=best_params['gb_n_estimators'])
xgb_clf = XGBClassifier(n_estimators=best_params['xgb_n_estimators'])

stacking_clf = StackingClassifier(estimators=[
    ('rf', rf_clf), 
    ('gb', gb_clf),
    ('xgb', xgb_clf)
], final_estimator=RandomForestClassifier(n_estimators=100))

# Cross-validation
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
y_test_proba = []

fig, ax = plt.subplots()

for i, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    X_train_cv, X_val_cv = X_train[train_index], X_train[val_index]
    y_train_cv, y_val_cv = y_train[train_index], y_train[val_index]

    stacking_clf.fit(X_train_cv, y_train_cv)
    y_val_pred = stacking_clf.predict(X_val_cv)
    y_val_proba = stacking_clf.predict_proba(X_val_cv)[:, 1]
    y_test_proba.append(stacking_clf.predict_proba(X_test)[:, 1])

    fpr, tpr, _ = roc_curve(y_val_cv, y_val_proba)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    ax.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', alpha=.8)
ax.set(xlim=[0, 1], ylim=[0, 1], title="Receiver Operating Characteristic")
ax.legend(loc="lower right")
plt.show()

# Final evaluation
stacking_clf.fit(X_train, y_train)
y_pred = stacking_clf.predict(X_test)

print(classification_report(y_test, y_pred))


346/346 ━━━━━━━━━━━━━━━━━━━━ 554s 2s/step


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


346/346 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
